# ДЗ №2 | Часть 2 | Adversarial Diffusion Distillation 


In [ ]:
!pip install -U diffusers --upgrade

In [ ]:
from tqdm.auto import tqdm

from copy import deepcopy
import csv
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import functools
from PIL import Image
from diffusers import StableDiffusionPipeline,UNet2DConditionModel, DDIMScheduler

from peft import LoraConfig, get_peft_model

%matplotlib inline
import matplotlib.pyplot as plt

torch.set_num_threads(16)

In [ ]:
#---------------------
# Visualization utils
#---------------------

def visualize_images(images):
    assert len(images) == 4
    plt.figure(figsize=(12, 3))
    for i, image in enumerate(images):
        plt.subplot(1, 4, i+1)
        plt.imshow(image)
        plt.axis('off')

    plt.subplots_adjust(wspace=-0.01, hspace=-0.01)


#--------------
# Tensor utils
#--------------

def extract_into_tensor(a, t, x_shape):
    b, *_ = t.shape
    out = a.gather(-1, t)
    return out.reshape(b, *((1,) * (len(x_shape) - 1)))

#---------------
# Dataset utils
#---------------

class TextImageDataset(torch.utils.data.Dataset):
    def __init__(self, root_dir, subset_name="train2014_5k", transform=None, max_cnt=None):
        """
        Arguments:
            root_dir (string): Директория с картинками
            transform (callable, optional): преобразования, применимые к картинкам
        """
        self.root_dir = root_dir
        self.transform = transform
        self.extensions = (
            ".jpg",
            ".jpeg",
            ".png",
            ".ppm",
            ".bmp",
            ".pgm",
            ".tif",
            ".tiff",
            ".webp",
        )
        sample_dir = os.path.join(root_dir, subset_name)

        # Собираем пути до картинок
        self.samples = sorted(
            [
                os.path.join(sample_dir, fname)
                for fname in os.listdir(sample_dir)
                if fname[-4:] in self.extensions
            ],
            key=lambda x: x.split("/")[-1].split(".")[0],
        )
        self.samples = (
            self.samples if max_cnt is None else self.samples[:max_cnt]
        )  # 

        # Собираем промпты
        self.captions = {}
        with open(
            os.path.join(root_dir, f"{subset_name}.csv"), newline="\n"
        ) as csvfile:
            spamreader = csv.reader(csvfile, delimiter=",")
            for i, row in enumerate(spamreader):
                if i == 0:
                    continue
                self.captions[row[1]] = row[2]

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        sample_path = self.samples[idx]
        sample = Image.open(sample_path).convert("RGB")

        if self.transform:
            sample = self.transform(sample)

        return {
            "image": sample,
            "text": self.captions[os.path.basename(sample_path)],
            "idxs": idx, }

# Модель учителя (SD1.5)

Для начала загрузим модель [StableDiffusion 1.5](https://huggingface.co/stable-diffusion-v1-5/stable-diffusion-v1-5) и сгенерируем ей картинки за 50 шагов.

**Важно:** для экономии памяти, загружаем все компоненты модели в FP16. Не забываем положить модель на GPU.

In [ ]:
pipe = <YOUR CODE HERE>

# Проверяем, что все компоненты модели в FP16 и на cuda
assert pipe.unet.dtype == torch.float16 and pipe.unet.device.type == 'cuda'
assert pipe.vae.dtype == torch.float16 and pipe.vae.device.type == 'cuda'
assert pipe.text_encoder.dtype == torch.float16 and pipe.text_encoder.device.type == 'cuda'

# Заменяем дефолтный сэмплер на DDIM
pipe.scheduler = DDIMScheduler.from_config(pipe.scheduler.config, timestep_spacing="trailing")
pipe.scheduler.timesteps = pipe.scheduler.timesteps.cuda()
pipe.scheduler.alphas_cumprod = pipe.scheduler.alphas_cumprod.cuda()

# Отдельно извлечем модель учителя, которую потом будем дистиллировать
teacher_unet = pipe.unet
teacher_unet.requires_grad_(False);

##  Создаем датасет

Мы будем дообучать модель на небольшой обучающей выборке из 10000 пар текст-картинка сгенерированные моделью [FLUX](https://huggingface.co/black-forest-labs/FLUX.1-dev).

Данные можно загрузить с помощью команд в ячейке ниже. В текущей директории ./ должны появиться:
* Папка flux_data с 10000 картинками
* Файл flux_data.csv с 10000 промптами

Данные парсятся корректным образом в уже реализованном классе **TextImageDataset**.

In [ ]:
!wget https://storage.yandexcloud.net/yandex-research/flux_data_10k.tar.gz
!tar -xzf flux_data_10k.tar.gz

In [ ]:
from torchvision import transforms

transform = transforms.Compose(
    [
        transforms.Resize(512),
        transforms.CenterCrop(512),
        transforms.ToTensor(),
        lambda x: 2 * x - 1,
    ]
)
dataset = TextImageDataset(".",
    subset_name="flux_data",
    transform=transform,
    max_cnt=5000 # Для дебага лучше взять 1000 или меньше
)

batch_size = 8 # Рекоммендуемы размер батча на Colab

train_dataloader = torch.utils.data.DataLoader(
    dataset=dataset, shuffle=True, batch_size=batch_size, drop_last=True
)

In [ ]:
@torch.no_grad()
def prepare_batch(batch, pipe):
    """
    Preprocess a batch of textual prompts and images to corresponding embeddings,
    using the text encoder and VAE
    Params:
    
    Return:
        latents: torch.Tensor([B, 4, 64, 64], dtype=torch.float16)
        prompt_embeds: torch.Tensor([B, 77, D], dtype=torch.float16)
    """
    
    # Tokenize prompts
    text_inputs = pipe.tokenizer(
        batch['text'],
        padding="max_length",
        max_length=pipe.tokenizer.model_max_length,
        truncation=True,
        return_tensors="pt",
    )
    
    # Exctract prompt embeddings using the text encoder
    prompt_embeds = pipe.text_encoder(text_inputs.input_ids.cuda())[0]

    # Map images to the VAE latent space
    image = batch['image'].to("cuda", dtype=torch.float16)
    latents = pipe.vae.encode(image).latent_dist.sample()
    latents = latents * pipe.vae.config.scaling_factor
    return latents, prompt_embeds

### Подготовка дискриминатора (1 балл)

Дискриминатор у нас будет классификационной головой поверх фичей учителя, как в [LADD](https://arxiv.org/abs/2403.12015). 

* Фичи мы будем извлекать с помощью forward hook-ов в торче. Функции хуков вам даны ниже. Вам нужно корректно применить их. Извлекать фичи будем из mid-block-a UNet-a.

* Архитектура MLP головы вам уже дана для экономии времени. FYI, она не обязательная должна быть именно такая. Просто мы взяли её из наших реализаций.   

* В классе дискриминатора нужно эту голову корректно применить к извлеченным фичам

In [ ]:
def save_tensors(module: nn.Module, features, name: str):
    """ Обработка и сохранение активаций в моделе """
    if type(features) in [list, tuple]:
        features = [f.float() if f is not None else None 
                    for f in features]
        setattr(module, name, features)
    elif isinstance(features, dict):
        features = {k: f.float() for k, f in features.items()}
        setattr(module, name, features)
    else:
        setattr(module, name, features.float())

        
def save_out_hook(self, inp, out):
    """ Cохраняет входы в модуль """
    save_tensors(self, out, 'activations')
    return out


def save_input_hook(self, inp, out):
    """ Cохраняет выходы из модуля """
    save_tensors(self, inp[0], 'activations')
    return out

In [ ]:
def FeedForward(dim, outdim=None, mult=1):
    outdim = dim if outdim is None else outdim
    return nn.Sequential(
        nn.LayerNorm(dim),
        nn.GELU(),
        nn.Linear(dim, outdim),
    )


class Discriminator(nn.Module):
    def __init__(self, teacher_unet, num_discriminator_layers=4):
        super().__init__()
        self.unet = teacher_unet

        dimensions = torch.linspace(
            teacher_unet.mid_block.attentions[0].norm.weight.shape[0],
            1,
            num_discriminator_layers + 1,
            dtype=int
        )
        
        # Создаем классификационную голову для дискриминатора
        self.list_of_layers = []
        for j, dim in enumerate(dimensions[:-1]):
            self.list_of_layers.append(FeedForward(dim.item(), dimensions[j+1].item()))
            
        self.cls_pred_branch = nn.Sequential(*self.list_of_layers)
        
        num_cls_params = sum(p.numel() for p in self.cls_pred_branch.parameters())
        print(f'Classification head number of trainable params: {num_cls_params}')
        
        # Зарегистрируйте форвард хук
        <YOUR CODE HERE>
        
    def forward(self, *args, **kwargs):
        # Прогоняем модель учителя, чтобы извлечь фичи
        self.unet(*args, **kwargs)
        
        # Извлекаем фичи и вычисляем логиты. Важно конвертировать фичи из FP16 в FP32
        <YOUR CODE HERE>
        return logits

In [ ]:
unet = UNet2DConditionModel.from_pretrained(
    'sd-legacy/stable-diffusion-v1-5', subfolder="unet",
).cuda().train()

assert unet.dtype == torch.float32
assert unet.training

In [ ]:
# Указываем к каким модулям модели мы будет добавлять адаптеры.
lora_modules = [
    "to_q", "to_k", "to_v", "to_out.0", "proj_in", "proj_out",
    "ff.net.0.proj", "ff.net.2", "conv1", "conv2", "conv_shortcut",
    "downsamplers.0.conv", "upsamplers.0.conv", "time_emb_proj"
]

lora_config = LoraConfig(
    r=64, # задает ранг у матриц A и B в LoRA.
    target_modules=lora_modules
)


# Создаем обертку исходной UNet модели с LoRA адаптерами, используя библиотеку PEFT
unet = get_peft_model(unet, lora_config, adapter_name="student")
optimizer = torch.optim.AdamW(unet.parameters(), lr=1e-4)


# Создаем дискриминатор и соотвествующий оптимизатор
D_unet = Discriminator(teacher_unet).cuda()
D_optimizer = torch.optim.AdamW(D_unet.parameters(), lr=1e-4)


# Включаем gradient checkpointing - важная техника для экономии памяти во время обучения
unet.enable_gradient_checkpointing()
teacher_unet.enable_gradient_checkpointing()

## Adversarial Diffusion Distillation (1 балл + 1 балл бонусом)

Здесь вам нужно реализовать ГАН лосс и функцию для предиктов студентом. ГАН лосс может быть любым на ваш вкус.

После этого нужно дописать цикл обучения

In [ ]:
def gan_loss_fn(fake_logits, real_logits=None):
    """ GAN лосс для дискриминатора и генератора """
    if real_logits is not None:
        # Discriminator loss
        <YOUR CODE HERE>
    else:
        # Generator loss
        <YOUR CODE HERE>
    return loss

In [ ]:
def get_x0_from_noise(sample, model_output, alphas_cumprod, timestep):
    """ Получение х_0 из x_t и предсказанного шума """
    <YOUR CODE HERE>
    return pred_original_sample

In [ ]:
def student_prediction(
    latents,
    prompt_embeds,
    unet, 
    scheduler,
    num_timesteps=1000,
    student_timesteps=[249, 499, 749, 999],
):
    # Сэмплируем x_t для студента из student_timesteps
    if isinstance(student_timesteps, list):
        student_timesteps = torch.tensor(student_timesteps, device=latents.device)
        
    index = torch.randint(
        0, len(student_timesteps), 
        (len(latents),), 
        device=latents.device
    ).long()
    
    timesteps = student_timesteps[index]
    
    # Получаем зашумленные латенты
    noisy_latents = <YOUR CODE HERE>

    # Сэмплируем студентом 
    # with <YOUR CODE HERE>: # для реализации mixed-precision обучения
    noise_pred = <YOUR CODE HERE>
    
    x0_pred = <YOUR CODE HERE>
    return x0_pred

### Обучающий цикл

Вам дан код обучения модель в полной точности (FP32) c батчом 8. В ячейке ниже вам нужно добавить mixed precision FP16/FP32 по аналогии с другими домашками. **Обратите внимание, что mixed-precision нужен только для обновления генератора**

Про реализацию mixed-precision в pytorch можно перейти по ссылке: [Mixed-precision обучение](https://pytorch.org/docs/stable/notes/amp_examples.html#typical-mixed-precision-training)

In [ ]:
torch.cuda.empty_cache()

discriminator_iters = 1

for i, batch in enumerate(tqdm(train_dataloader)):
    
    # Сэмплируем батч из датасета
    latents, prompt_embeds = prepare_batch(batch, pipe)
    
    # Обновляем дискриминатор
    D_unet.cls_pred_branch.requires_grad_(True) # Включаем градиенты до головы
    for _ in range(discriminator_iters):
        with torch.no_grad():
            fake_latents = <YOUR CODE HERE>
        
        # Вычисляем логиты из дискриминатора. 
        # Не забудьте, что учитель внутри дискриминатора в FP16
        # Для дискриминатора будем использова t=0
        D_timesteps = torch.zeros(len(latents), device=latents.device)
        
        fake_logits = <YOUR CODE HERE>

        real_logits = <YOUR CODE HERE>
        
        gan_cls_loss = <YOUR CODE HERE>
        
        # Обновляем параметры fake модели
        gan_cls_loss.backward()
        D_optimizer.step()
        D_optimizer.zero_grad(set_to_none=True)
    
    
    # Сэмплируем латенты для обновления студента. Уже без torch.no_grad()
    fake_latents = <YOUR CODE HERE>
    
    D_unet.cls_pred_branch.requires_grad_(False) # Выключаем градиенты до головы
    fake_logits = <YOUR CODE HERE>
    gan_gen_loss = <YOUR CODE HERE>
    
    # Обновляем параметры
    gan_gen_loss.backward()
    optimizer.step()
    optimizer.zero_grad(set_to_none=True)
    
    print(f"G loss: {gan_gen_loss.item():.3f} | D loss: {gan_cls_loss.item():.3f}")

### Генерация с помощью multistep stochastic sampling

Генерируем картинки с помощью нашей модели. Нам нужен специальный сэмплер, который схематично изображен на картинке ниже. 

**Эту функцию вам нужно взять из Части 1. Там она называется consistency_sampling.**

<div>
<img src="https://i.postimg.cc/66bWLvnh/cd-sampling.jpg" width="600"/>
</div>

Чуть более формально:

$x_{t_n} \sim \mathcal{N}(0, I)$

$for\ t_i \in [t_n, ..., t_1]:$

* $\epsilon \leftarrow unet(x_{t_i})$

* $x_0 \leftarrow DDIM(\epsilon, x_{t_i}, t_i, 0)$

* $x_{t_{i-1}} \leftarrow q(x_{t_{i-1}} | x_0)$


In [ ]:
# СОВПАДАЕТ С consistency_sampling ИЗ ЧАСТИ 1. МОЖНО СКОПИРОВАТЬ ОТТУДА

@torch.no_grad()
def multistep_sampling(
    pipe,
    prompt,
    num_inference_steps=4,
    generator=None,
    num_images_per_prompt=4,
    guidance_scale=1
):
    if prompt is not None and isinstance(prompt, str):
        batch_size = 1
    elif prompt is not None and isinstance(prompt, list):
        batch_size = len(prompt)

    device = pipe._execution_device

    # Извлекаем эмбеды из текстовых промптов. Реализуйте вызов pipe.encode_prompt
    do_classifier_free_guidance = guidance_scale > 0
    prompt_embeds, null_prompt_embeds = <YOUR CODE HERE>
    assert prompt_embeds.dtype == null_prompt_embeds.dtype == torch.float16

    # Настраиваем параметры scheduler-a
    assert pipe.scheduler.config['timestep_spacing'] == 'trailing'
    pipe.scheduler.set_timesteps(num_inference_steps)

    # Создаем батч латентов из N(0,I)
    latents = <YOUR CODE HERE>

    for i, t in enumerate(tqdm(pipe.scheduler.timesteps)):
        t = torch.tensor([t] * len(latents)).to(device)
        zero_t = torch.tensor([0] * len(latents)).to(device)

        cond_noise_pred = <YOUR CODE HERE>

        if do_classifier_free_guidance:
            uncond_noise_pred = <YOUR CODE HERE>
            noise_pred = <YOUR CODE HERE>
        else:
            noise_pred = cond_noise_pred

        # Получаем x_0 оценку из x_t
        x_0 = <YOUR CODE HERE>

        if i + 1 < num_inference_steps:
            # Переход на следующий шаг
            s = pipe.scheduler.timesteps[i+1]
            s = torch.tensor([s] * len(latents)).to(device)

            latents = <YOUR CODE HERE>
        else:
            # Последний шаг
            latents = x_0

        latents = latents.half()

    # Декодируем латенты в пиксели. Не забудьте про pipe.vae.config.scaling_factor 
    image = <YOUR CODE HERE>
    do_denormalize = [True] * image.shape[0]
    image = pipe.image_processor.postprocess(image, output_type="pil", do_denormalize=do_denormalize)
    return image

### Сгенерируем картинки для разных промптов

In [ ]:
validation_prompts = [
    "A sad puppy with large eyes",
    "A girl with pale blue hair and a cami tank top",
    "A lighthouse in a giant wave, origami style",
    "belle epoque, christmas, red house in the forest, photo realistic, 8k",
    "A small cactus with a happy face in the Sahara desert",
]

In [ ]:
pipe.unet = unet.eval().to(torch.float16)
assert unet.active_adapter == 'student'

guidance_scale = 1

for prompt in validation_prompts:
    generator = torch.Generator(device="cuda").manual_seed(1)

    # Применяем консистенси сэмплирование.
    images = <YOUR CODE HERE>

    visualize_images(images)

**Важно:** ваши результаты не должны совпадать с теми что ниже. Это скорее ориентир по качеству. Вполне вероятно у вас могут получиться картинки сильно лучше :)

**Референсные примеры CFG=1**

![img](https://storage.yandexcloud.net/yandex-research/ysda-hw2-add-references/reference_add_dog.png)

![img](https://storage.yandexcloud.net/yandex-research/ysda-hw2-add-references/reference_add_girl_cfg1.png)

![img](https://storage.yandexcloud.net/yandex-research/ysda-hw2-add-references/reference_add_house_cfg1.png)

![img](https://storage.yandexcloud.net/yandex-research/ysda-hw2-add-references/reference_add_lighthouse_cfg1.png)


**Референсные примеры CFG=2**

![img](https://storage.yandexcloud.net/yandex-research/ysda-hw2-add-references/reference_add_dog_cfg2.png)

![img](https://storage.yandexcloud.net/yandex-research/ysda-hw2-add-references/reference_add_girl_cfg2.png)

![img](https://storage.yandexcloud.net/yandex-research/ysda-hw2-add-references/reference_add_house_cfg2.png)

![img](https://storage.yandexcloud.net/yandex-research/ysda-hw2-add-references/reference_add_lighthouse_cfg2.png)